In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [2]:
train_df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

# Quesiton 1

In [3]:
train_df.shape

(3066766, 19)

In [4]:
train_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Question 2

In [5]:
train_df['duration'] = train_df.tpep_dropoff_datetime - train_df.tpep_pickup_datetime
train_df.duration = train_df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
train_df.duration.describe().round(2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

# Question 3

In [7]:
train_outlier_duration_cond = (train_df.duration >= 1) & (train_df.duration <= 60)
print("% of records that are not outliers:", np.round(train_outlier_duration_cond.sum()/len(train_df)*100, 2))

% of records that are not outliers: 98.12


In [8]:
train_df = train_df[train_outlier_duration_cond]

# Question 4
## Turn the dataframe into a list of dictionaries

In [9]:
categorical = ['PULocationID', 'DOLocationID']
train_df[categorical] = train_df[categorical].astype(str)
train_dicts = train_df[categorical].to_dict(orient='records')

My question: why is it better to turn the dataframe into a list of dictionaries/vectorize it? Is it for speed/efficiency?

## Fit a dictionary vectorizer

In [28]:
dv = DictVectorizer()

## Get a feature matrix from it

In [ ]:
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train.shape

(3009173, 515)

# Question 5
## Train a plain linear regression model with default parameters, where duration is the response variable


In [13]:
target = 'duration'
y_train = train_df[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = lr.predict(X_train)

## Calculate the RMSE of the model on the training data

In [16]:
mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

# Question 6
## What's the RMSE on validation?

In [17]:
# Putting together the code from before
def get_clean_df(path):
    # grab data
    df = pd.read_parquet(path)

    # create duration feature
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # drop outliers
    outlier_duration_cond = (df.duration >= 1) & (df.duration <= 60)
    df = df[outlier_duration_cond]

    return df

def get_target_feature(df):
    return df['duration'].values

def predict_lr(lr, X):
    return lr.predict(X)

def get_rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

In [18]:
test_df = get_clean_df('../data/yellow_tripdata_2023-02.parquet')

In [19]:
test_df.shape

(2855951, 20)

In [20]:
test_dicts = test_df[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [22]:
X_test.shape

(2855951, 515)

In [23]:
y_test = get_target_feature(test_df)

In [25]:
y_test_pred = predict_lr(lr, X_test)

In [26]:
get_rmse(y_test, y_test_pred)

13.314356413009067